In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2023
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
import os, warnings, sys, logging
import mlflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
import mlflow.sklearn
from xgboost import XGBClassifier
from datetime import date
import cml.data_v1 as cmldata
import pyspark.pandas as ps

import onnxmltools
from urllib.parse import urlparse
from sklearn.preprocessing import FunctionTransformer
from mlflow.models import infer_signature
from onnxconverter_common import FloatTensorType

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [3]:
USERNAME = os.environ["PROJECT_OWNER"]
DBNAME = "BNK_MLOPS_HOL_"+USERNAME
CONNECTION_NAME = "go01-aw-dl"

DATE = date.today()
EXPERIMENT_NAME = "xgb-cc-fraud-{0}".format(USERNAME)

mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/cdsw/.experiments/nioc-34os-yy9s-k2jb', creation_time=None, experiment_id='nioc-34os-yy9s-k2jb', last_update_time=None, lifecycle_stage='active', name='xgb-cc-fraud-pauldefusco', tags={}>

In [4]:
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


Spark Application Id:spark-b39e5df9541d414f8400bed6c302228e


In [5]:
#df.columns

In [10]:
df_from_sql = ps.read_table('{0}.CC_TRX_{1}'.format(DBNAME, USERNAME))
df = df_from_sql.to_pandas()

y = df["fraud_trx"]
X = df.drop(columns=["job", "fraud_trx"])
X.columns = ['f' + str(i) for i in range(len(X.columns))]

test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_table`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/opt/spark/python/lib/pyspark.zip/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [11]:
with mlflow.start_run():

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")

    # Step 1: cambiar test_size linea 69 y recorrer
    # Step 2: cambiar linea 74, agregar linea 97, y recorrer
      # linea 75: model = XGBClassifier(use_label_encoder=False, max_depth=4, eval_metric="logloss")
      # linea 97: mlflow.log_param("max_depth", 4)
    # Step 3: cambiar linea 74 y 97, agregar linea 98, y recorrer
      # linea 75: model = XGBClassifier(use_label_encoder=False, max_depth=2, max_leaf_nodes=5, eval_metric="logloss")
      # linea 97: mlflow.log_param("max_depth", 2)
      # linea 98: mlflow.log_param("max_leaf_nodes", 5)

    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Test Size: %.2f%%" % (test_size * 100.0))

    mlflow.log_param("accuracy", accuracy)
    mlflow.log_param("test_size", test_size)

    # Step 2:
    # Step 3:

    num_features = X_train.shape[1]
    initial_type = [("input", FloatTensorType([None, num_features]))]

    model_signature = infer_signature(X_train, y_pred)
    onnx_model = onnxmltools.convert_xgboost(model, initial_types=initial_type)
    #onnxmltools.utils.save_model(onnx_model, "fraud_classifier.onnx")
    mlflow.onnx.log_model(onnx_model, "fraud-clf-onnx-xgboost",
                          registered_model_name="fraud-detector-onnx-xgboost",
                          signature=model_signature)

/home/cdsw/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:41:26] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 89.13%
Test Size: 30.00%


Successfully registered model 'fraud-detector-onnx-xgboost'.
2025/03/24 00:41:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: fraud-detector-onnx-xgboost, version 4
Created version '4' of model 'fraud-detector-onnx-xgboost'.
